# Audiobooks

# Preprocess

In [12]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing

raw_csv_data = np.loadtxt('../docs/Audiobooks_data.csv', delimiter=",")

unscaled_inputs_all = raw_csv_data[:, 1:-1]
targets_all = raw_csv_data[:,-1]

### Shuffle

In [13]:
shuffled_indices = np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffled_indices)

unscaled_inputs_all = unscaled_inputs_all[shuffled_indices]
targets_all = targets_all[shuffled_indices]

### Balance the dataset

In [14]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if  targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
unscaled_inputs_balanced = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_balanced = np.delete(targets_all, indices_to_remove, axis=0)

### Standardize

In [15]:
scaled_inputs = preprocessing.scale(unscaled_inputs_balanced)

### Shuffle

In [16]:
shuffle_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffle_indices)

shuffled_inputs = scaled_inputs[shuffle_indices]
shuffled_targets = targets_balanced[shuffle_indices]

### Traing - Validation - Test

In [17]:
samples_count = shuffle_indices.shape[0]

train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1804.0 3579 0.5040514110086617
210.0 447 0.4697986577181208
223.0 448 0.49776785714285715


In [18]:
np.savez( 'Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez( 'Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez( 'Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

# Machine learning algorithm

In [ ]:
npz = np.load('Audiobooks_data_train.npz')
train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs = npz['inputs'].astype(np.float)
validation_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs_inputs = npz['inputs'].astype(np.float)
test_inputs_targets = npz['targets'].astype(np.int)

### Model

In [20]:
input_size = 10
output_size = 2
hidden_layer_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax'),
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100
max_epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs, train_targets, batch_size=batch_size, epochs=max_epochs, callbacks=early_stopping, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/100
36/36 - 1s - loss: 0.5273 - accuracy: 0.7429 - val_loss: 0.4542 - val_accuracy: 0.8009 - 1s/epoch - 41ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4214 - accuracy: 0.7868 - val_loss: 0.4246 - val_accuracy: 0.7919 - 132ms/epoch - 4ms/step
Epoch 3/100
36/36 - 0s - loss: 0.3993 - accuracy: 0.7921 - val_loss: 0.3967 - val_accuracy: 0.8143 - 155ms/epoch - 4ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3878 - accuracy: 0.8036 - val_loss: 0.3904 - val_accuracy: 0.8121 - 156ms/epoch - 4ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3753 - accuracy: 0.8030 - val_loss: 0.3812 - val_accuracy: 0.8166 - 144ms/epoch - 4ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3727 - accuracy: 0.7955 - val_loss: 0.3815 - val_accuracy: 0.8210 - 134ms/epoch - 4ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3650 - accuracy: 0.8072 - val_loss: 0.3955 - val_accuracy: 0.8098 - 140ms/epoch - 4ms/step


# Testing

In [21]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3696 - accuracy: 0.8080
